# 작업 1유형

## 1-1
각 구급 보고서 별 출동시각과 신고시각의 차이를 ‘소요시간’ 컬럼을 만들고 초(sec)단위로 구하고 소방서명 별 소요시간의 평균을 오름차순으로 정렬 했을때 3번째로 작은 소요시간의 값과 소방서명을  출력하라

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_1.csv')
df.head(5)

,구급보고서번호,소방서명,환자증상구분명1,신고일자,신고시각,출동일자,출동시각,환자직업명,사고발생장소
0,20211122103M01526,중랑소방서,심계항진,20210330,104800,20210330,105042,무직,집
1,20211111105M00771,노원소방서,복통,20210412,225600,20210412,225827,기타,집
2,20211115104M00864,서초소방서,심계항진,20210303,194100,20210303,194500,직장인,상업시설
3,20211113107M01869,마포소방서,고열,20211225,235400,20211226,331,유아,집
4,20211118102M05820,강동소방서,고열,20211019,12100,20211019,12322,무직,집


In [2]:
df['출동일시'] = pd.to_datetime(df['출동일자'].astype(str) + df['출동시각'].astype(str).str.zfill(6))
df['신고일시'] = pd.to_datetime(df['신고일자'].astype(str) + df['신고시각'].astype(str).str.zfill(6))

df['소요시간'] = (df['출동일시'] - df['신고일시']).dt.total_seconds()

df.groupby(['소방서명'])['소요시간'].mean().sort_values().reset_index().iloc[2].to_list()

['종로소방서', 175.5]

## 1-2
학교 세부유형이 일반중학교인 학교들 중 일반중학교 숫자가 2번째로 많은 시도의 일반중학교 데이터만 필터하여 해당 시도의 교원 한명 당 맡은 학생수가 가장 많은 학교를 찾아서 해당 학교의 교원수를  출력하라

In [3]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_2.csv')
df.head(5)

,학교명,시도,학교세부유형,일반학급_학생수_계,교원수_총계_계
0,서울대학교사범대학부설고등학교,서울,일반고등학교,689,73
1,가락고등학교,서울,일반고등학교,765,70
2,개포고등학교,서울,일반고등학교,684,69
3,경기고등학교,서울,일반고등학교,1047,96
4,경동고등학교,서울,일반고등학교,529,58


In [4]:
trg_city = df[df['학교세부유형'] == '일반중학교']['시도'].value_counts().index[1]
cond1 = df['시도'] == trg_city
cond2 = df['학교세부유형'] == '일반중학교'
cond = cond1 & cond2
new_df = df[cond].reset_index(drop=True)
new_df['ratio'] = new_df['일반학급_학생수_계'] / new_df['교원수_총계_계']
new_df.sort_values(by='ratio', ascending=False)['교원수_총계_계'].values[0]

33

## 1-3
5대 범죄(절도, 사기, 배임, 방화, 폭행)의 월별 총 발생건수를 총범죄수라고 표현하자. 18,19년의 각각 분기별 총범죄수의 월평균 값을 구했을때 최대값을 가지는 년도와 분기를 구하고 해당 분기의 최댓값의 사기가 발생한 월의 사기 발생건수를  출력하라(1분기:1,2,3월 / 2분기 : 4,5,6월 / 3분기 7,8,9월 / 4분기 10,11,12월 , 1분기 월평균 : 1,2,3월의 총범죄수 평균)

In [5]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_3.csv')
df.head(5)

,범죄분류,절도,사기,배임,방화,폭행
0,2018년_1월,13280,25318,656,143,12186
1,2018년_2월,12504,19159,403,135,12087
2,2018년_3월,14733,24509,471,148,13841
3,2018년_4월,14781,23147,414,135,13973
4,2018년_5월,15988,23584,451,127,14117


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   범죄분류    24 non-null     object
 1   절도      24 non-null     int64 
 2   사기      24 non-null     int64 
 3   배임      24 non-null     int64 
 4   방화      24 non-null     int64 
 5   폭행      24 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.2+ KB


In [7]:
int_columns = df.select_dtypes(include=['int64']).columns

In [8]:
df['총범죄수'] = df[int_columns].sum(axis=1)

In [9]:
df['년'] = df['범죄분류'].map(lambda x : x.split('_')[0].replace('년', ''))
df['월'] = df['범죄분류'].map(lambda x : x.split('_')[1].replace('월', ''))

In [10]:
def get_quarter(x):
    if x in [1, 2, 3]:
        return '1분기'
    elif x in [4, 5, 6]:
        return '2분기'
    elif x in [7, 8, 9]:
        return '3분기'
    else:
        return '4분기'

df['분기'] = df['월'].astype(int).apply(get_quarter)

In [11]:
# 18,19년의 각각 분기별 총범죄수의 월평균 값을 구했을때 최대값을 가지는 년도와 분기를 구하고
max_year, max_quarter, max_crime_number = df.groupby(by=['년', '분기'], as_index=False)['총범죄수'].mean().sort_values(by='총범죄수').iloc[-1]
max_year, max_quarter, max_crime_number

('2019', '2분기', 58438.333333333336)

In [12]:
# 해당 분기의 최댓값의 사기가 발생한 월의 사기 발생건수를 출력하라
df[(df['년'] == max_year) & (df['분기'] == max_quarter)].sort_values(by='사기').iloc[-1]['사기']

27766

# 작업 2유형
예측 변수 General_Health, test.csv에 대해 ID별로 General_Health 값을 예측하여 제출, 제출 데이터 컬럼은 ID와 General_Health 두개만 존재해야함. 평가지표는 f1score

In [164]:
import pandas as pd
pd.set_option('display.max_columns', None)  # 모든 컬럼 표시
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [165]:
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_test.csv')

### sol1

In [157]:
# 데이터 분리

X = train.drop(['ID', 'General_Health'], axis=1)
y = train['General_Health']
test_id = test.pop('ID')

In [158]:
# 원핫 인코딩
X = pd.get_dummies(X)
test = pd.get_dummies(test)

In [159]:
X_train, X_val, y_train, y_val = train_test_split(X, y)

In [160]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [161]:
pred_val = model.predict(X_val)
print('val f1score', f1_score(y_val, pred_val, average='macro'))

val f1score 0.5443674350048754


In [162]:
pred_test = model.predict(test)

In [163]:
sumission = pd.DataFrame()
sumission['ID'] = test_id
sumission['price'] = pred_test
sumission.to_csv('00000000.csv', index=False)

### sol2

In [166]:
import pandas as pd
pd.set_option('display.max_columns', None)  # 모든 컬럼 표시
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [167]:
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/ep6_p2_test.csv')

In [168]:
train.head(2)

,ID,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,ID_1473,Good,Within the past 2 years,Yes,No,No,No,No,No,Yes,Female,55-59,168.0,77.11,27.44,No,1.0,12.0,4.0,4.0
1,ID_2045,Good,Within the past 2 years,Yes,No,No,No,No,No,No,Female,40-44,152.0,58.97,25.39,No,0.0,8.0,16.0,8.0


In [169]:
test.head(2)

,ID,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,ID_14547,Within the past year,Yes,No,No,No,No,No,No,Male,35-39,180.0,81.65,25.10,No,15.0,8.0,25.0,12.0
1,ID_26,5 or more years ago,No,No,Yes,No,No,No,No,Male,60-64,183.0,79.38,23.73,No,1.0,4.0,4.0,4.0


In [170]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11230 entries, 0 to 11229
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            11230 non-null  object 
 1   General_Health                11230 non-null  object 
 2   Checkup                       11230 non-null  object 
 3   Exercise                      11230 non-null  object 
 4   Heart_Disease                 11230 non-null  object 
 5   Skin_Cancer                   11230 non-null  object 
 6   Other_Cancer                  11230 non-null  object 
 7   Depression                    11230 non-null  object 
 8   Diabetes                      11230 non-null  object 
 9   Arthritis                     11230 non-null  object 
 10  Sex                           11230 non-null  object 
 11  Age_Category                  11230 non-null  object 
 12  Height_(cm)                   11230 non-null  float64
 13  W

In [171]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4813 entries, 0 to 4812
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            4813 non-null   object 
 1   Checkup                       4813 non-null   object 
 2   Exercise                      4813 non-null   object 
 3   Heart_Disease                 4813 non-null   object 
 4   Skin_Cancer                   4813 non-null   object 
 5   Other_Cancer                  4813 non-null   object 
 6   Depression                    4813 non-null   object 
 7   Diabetes                      4813 non-null   object 
 8   Arthritis                     4813 non-null   object 
 9   Sex                           4813 non-null   object 
 10  Age_Category                  4813 non-null   object 
 11  Height_(cm)                   4813 non-null   float64
 12  Weight_(kg)                   4813 non-null   float64
 13  BMI

In [172]:
# train 컬럼 유니크 값
for col in train.columns:
    print(f'unique of {col} :', np.sort(train[col].unique()))

unique of ID : ['ID_1' 'ID_10' 'ID_100' ... 'ID_9995' 'ID_9996' 'ID_9999']
unique of General_Health : ['Excellent' 'Fair' 'Good']
unique of Checkup : ['5 or more years ago' 'Never' 'Within the past 2 years'
 'Within the past 5 years' 'Within the past year']
unique of Exercise : ['No' 'Yes']
unique of Heart_Disease : ['No' 'Yes']
unique of Skin_Cancer : ['No' 'Yes']
unique of Other_Cancer : ['No' 'Yes']
unique of Depression : ['No' 'Yes']
unique of Diabetes : ['No' 'No, pre-diabetes or borderline diabetes' 'Yes'
 'Yes, but female told only during pregnancy']
unique of Arthritis : ['No' 'Yes']
unique of Sex : ['Female' 'Male']
unique of Age_Category : ['18-24' '25-29' '30-34' '35-39' '40-44' '45-49' '50-54' '55-59' '60-64'
 '65-69' '70-74' '75-79' '80+']
unique of Height_(cm) : [ 94. 102. 105. 106. 115. 122. 127. 135. 137. 140. 142. 144. 145. 146.
 147. 150. 152. 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164.
 165. 166. 167. 168. 170. 171. 172. 173. 175. 177. 178. 180. 181. 182.


In [21]:
# test 컬럼 유니크 값
for col in test.columns:
    print(f'unique of {col} :', np.sort(test[col].unique()))

unique of ID : ['ID_10001' 'ID_10005' 'ID_10006' ... 'ID_9994' 'ID_9997' 'ID_9998']
unique of Checkup : ['5 or more years ago' 'Never' 'Within the past 2 years'
 'Within the past 5 years' 'Within the past year']
unique of Exercise : ['No' 'Yes']
unique of Heart_Disease : ['No' 'Yes']
unique of Skin_Cancer : ['No' 'Yes']
unique of Other_Cancer : ['No' 'Yes']
unique of Depression : ['No' 'Yes']
unique of Diabetes : ['No' 'No, pre-diabetes or borderline diabetes' 'Yes'
 'Yes, but female told only during pregnancy']
unique of Arthritis : ['No' 'Yes']
unique of Sex : ['Female' 'Male']
unique of Age_Category : ['18-24' '25-29' '30-34' '35-39' '40-44' '45-49' '50-54' '55-59' '60-64'
 '65-69' '70-74' '75-79' '80+']
unique of Height_(cm) : [100. 102. 124. 127. 135. 140. 142. 145. 147. 149. 150. 152. 155. 156.
 157. 158. 159. 160. 163. 164. 165. 167. 168. 170. 172. 173. 174. 175.
 176. 178. 180. 183. 185. 187. 188. 191. 193. 196. 198. 201. 203. 205.
 206. 208.]
unique of Weight_(kg) : [ 36.29  3

# 컬럼 요약
- ID : ID
- General_Health : 타겟 데이터. 범주형-순서형. 레이블 인코딩
- Checkup : 범주형-명목형. 원핫 인코딩
- Exercise : 범주형-명목형. 원핫 인코딩
- Heart_Disease : 범주형-명목형. 원핫 인코딩
- Skin_Cancer : 범주형-명목형. 원핫 인코딩
- Other_Cnacer : 범주형-명목형. 원핫 인코딩
- Depression : 범주형-명목형. 원핫 인코딩
- Diabetes : 범주형-명목형. 원핫 인코딩
- Arthritis : 범주형-명목형. 원핫 인코딩
- Sex : 범주형-명목형. 원핫 인코딩
- Age_Category : 범주형-순서형. 숫자 매핑
- Height_(cm) : 수치형-연속형. 스케일링
- Weight_(kg) : 수치형-연속형. 스케일링
- BMI : 수치형-연속형. 스케일링
- Smoking_History : 범주형-명목형. 원핫 인코딩
- Alcohol_Consumption : 수치형-이산형.
- Fruit_Consumption : 수치형-이산형
- Green_Vegetables_Consumption : 수치형-이산형
- FriedPotato_Consumption : 수치형-이산형

In [173]:
# 원핫 인코딩
oh_cols = ['Checkup', 'Exercise', 'Heart_Disease', 'Skin_Cancer', 'Other_Cancer', 'Depression', 'Diabetes', 'Arthritis', 'Sex', 'Smoking_History']
train = pd.get_dummies(train, columns=oh_cols)
test = pd.get_dummies(test, columns=oh_cols)

In [174]:
# 숫자 매핑 - Age_Category
age_category = np.sort(train['Age_Category'].unique())
age_dic = {}

for i, age in enumerate(age_category):
    age_dic[age] = i

train['Age_Category'] = train['Age_Category'].map(age_dic)
test['Age_Category'] = test['Age_Category'].map(age_dic)

In [175]:
# 데이터 분리
X = train.drop(['ID', 'General_Health'], axis=1)
y = train['General_Health']
test_id = test.pop('ID')

In [176]:
# 학습데이터 / 검증데이터 분리

random_state=42
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=random_state)

In [177]:
# 모델 입력

model = RandomForestClassifier(random_state=random_state)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [181]:
# 검증 데이터 추론
pred_val = model.predict(X_val)

In [182]:
# 검증 결과 평가
from sklearn.metrics import f1_score
ans = f1_score(y_val, pred_val, average='micro')
print('f1-score:', ans)

f1-score: 0.5587711487088157


In [183]:
# test 데이터 추론
pred_test = model.predict(test)
pred_test

array(['Excellent', 'Excellent', 'Good', ..., 'Fair', 'Good', 'Good'],
      dtype=object)

In [185]:
# 제출
submission = pd.DataFrame({'ID': test_id, 'General_Health': pred_test})
submission.to_csv('submission.csv', index=False)

# 작업 3유형
A 도시의 남성 600명과 여성 550명이 있다. 남성들 중 흡연자 비율은 0.2이며 여성들 중 흡연자 비율은 0.26이다.
남성과 여성 간에 흡연 여부에 따른 인구 비율이 다른지 확인하고 싶다. 유의 수준 0.05하 귀무가설에 대해 기각 / 채택 여부와 p-value값을 각각  출력하라

In [189]:
import pandas as pd 
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/28/p7.csv')
df.head()

,age,Cholesterol,weight
0,65,69.0,111.0
1,54,117.0,81.0
2,61,86.2,72.0
3,57,76.0,78.0
4,62,160.0,61.0
